**Rust Quantum Library**

Below are dependencies, creates, and global variables.

In [2]:
:dep ndarray = { version = "0.14", features = ["serde"] }
:dep ndarray-linalg = { version = "0.13.1", features = ["intel-mkl-static"]}
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series"] }
:dep thiserror = "1.0"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep dimensioned = "0.7"
:dep num-complex = "0.4"
:dep qm = { version = "0.1", git ="https://github.com/buff-beacon-project/qm" }

extern crate plotters;
extern crate ndarray;

pub use ndarray_linalg::c64;
use std::f64::consts::PI;
use plotters::prelude::*;
use ndarray::prelude::*;
use ndarray_linalg::*;
use qm::*;

pub type VecC64 = ndarray::Array1<c64>;
pub type MatrixC64 = ndarray::Array2<c64>;

pub const NORM_CONST: f64 = 1./std::f64::consts::SQRT_2;

*The Density Matrix*

The density matrix is a more general way of characterizing the overall state of a quantum system.  It is represented as $\rho = |\psi\rangle\langle\psi|$, where the wavefunction $|\psi\rangle$ is represented in Dirac notation as a column vector, and $\langle\psi|$ is the complex conjugate of the row vector. The density matrix can describe both pure states and mixed states. Pure states represent the state of one particle or the average state of an ensemble. In reality, most of the time mixed states are prepared or show up in nature. For example, light from the sun is a full mixture of quantum states, so it is not unpolarized and considered a maximally mixed state.

Here, we create density matrices out of the four Bell States, which are entangled quantum states:

$$\phi_- = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$$
$$\phi_+ = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$$
$$\psi_- = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$$
$$\psi_+ = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$$

In [ ]:
let bell_phi_plus_vec: VecC64 = array![c64::new(NORM_CONST , 0.0) , c64::new(0.0 , 0.0) , c64::new(0.0 , 0.0) , c64::new(NORM_CONST , 0.0) ];

let bell_phi_minus_vec: VecC64 = array![c64::new(NORM_CONST , 0.0) , c64::new(0.0 , 0.0) , c64::new(0.0 , 0.0) , c64::new(-NORM_CONST , 0.0) ];

let bell_psi_plus_vec: VecC64 = array![c64::new(0.0 , 0.0) , c64::new(NORM_CONST , 0.0) , c64::new(NORM_CONST , 0.0) , c64::new(0.0 , 0.0) ];

let bell_psi_minus_vec: VecC64 = array![c64::new(0.0 , 0.0) , c64::new(NORM_CONST , 0.0) , c64::new(-NORM_CONST , 0.0) , c64::new(0.0 , 0.0) ];

let rho_phi_plus = create_dens_matrix(&bell_phi_plus_vector);
let rho_phi_minus = create_dens_matrix(&bell_phi_minus_vector);
let rho_psi_plus = create_dens_matrix(&bell_psi_plus_vector);
let rho_psi_minus = create_dens_matrix(&bell_psi_minus_vector);

println!("Density matrix for phi_+ = \n {:.1}", rho_phi_plus );
println!("Density matrix for phi_- = \n {:.1}", rho_phi_minus );
println!("Density matrix for psi_+ = \n {:.1}", rho_psi_plus );
println!("Density matrix for psi_- = \n {:.1}", rho_psi_minus );

*Purity*

Purity is the trace of the square of the density matrix, $Tr(\rho^{2})$. In this example, we continuously add mixtures to a pure Bell state and graph its purity. The purity ranges from $\frac{1}{d}$ to 1, where $d$ is the dimension of the density matrix. In this case with a 4x4 density matrices, the purity ranges from 0.25 to 1.

In [ ]:
let bell_phi_plus_coef: VecC64 = array![c64::new(norm_const , 0.0) , c64::new(0.0        , 0.0) , 
                                        c64::new(0.0        , 0.0) , c64::new(norm_const , 0.0) ];

let mixed_state: MatrixC64 = array![ [  c64::new(0.25 , 0.) ,  c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0. , 0.)  ] , 
                                  [  c64::new(0. , 0.) ,   c64::new(0.25 , 0.) , c64::new(0. , 0.)  ,  c64::new(0. , 0.)   ] ,
                                  [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0.25 , 0.) ,  c64::new(0. , 0.)   ] ,
                                  [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0.25 , 0.)   ] ];

let mut points: Vec<(f64, f64)> = Vec::new();

for i in 0..100{
    let eta = c64::new((i as f64)/100. , 0.); 
    let mut mixture = (c64::new(1. , 0.)-eta)*create_dens_matrix(&bell_phi_plus_coef) + eta*mixed_state.clone();
    let mut mixture_sqrd = find_matrix_sqrd(mixture);
    
    let mut purity = find_purity(&mixture_sqrd);
    points.push( (eta.re, purity) );
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Purity of partially mixed Bell state", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..1.1, 0_f64..1.01)?;
    
    chart
        .configure_mesh()
        .x_desc("eta")
        .y_desc("Purity")
        .draw()?;
    chart.draw_series(points.into_iter().map(|(x,y)| Circle::new((x,y), 3, BLUE.filled())));
    Ok(())
});

figure

*Concurrence*

In [ ]:
let mut points: Vec<(f64, f64)> = Vec::new();
let mut points_x: Vec<f64> = Vec::new();
let mut points_y: Vec<f64> = Vec::new();

for i in 0..360{
    let mut theta: f64 = (i as f64)*PI/180.;
    let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
    let mut rho_part_entangled: MatrixC64 = create_dens_matrix(&psi_part_entangled);
    let mut concurrence: f64 = find_concurrence(rho_part_entangled);
    points.push( (theta, concurrence) );
    points_x.push(theta);
    points_y.push(concurrence);
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Concurrence of partially entangled state over theta", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..2.*PI, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("theta")
        .y_desc("Concurrence")
        .draw()?;
    
    chart.draw_series(LineSeries::new(
        points_x.into_iter().zip(points_y.into_iter()), &RED,))?;

//     chart.configure_series_labels()
//         .background_style(&WHITE.mix(0.8))
//         .border_style(&BLACK)
//         .draw()?;
    Ok(())
});
figure

*Concurrence - adding in a mixed state*

In this example, the 2D rectangular plot shows $\theta$ on the x axis and $\eta$ on the y axis. The concurrence is plotted as a function of $\eta\rho_{pure} + (1-\eta)\rho_{mixed}$. The pure density matrix is the density matrix of the partially entangled state $\psi = \sin\theta|00\rangle + \cos\theta|11\rangle$, and $\rho_{mixed} = 0.25I$ where $I$ is the 4x4 idenity matrix.

In [ ]:
let mut points: Vec<(f64, f64, f64)> = Vec::new();

let mixed_state: MatrixC64 = array![ [  c64::new(0.25 , 0.) ,  c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0. , 0.)  ] , 
                                     [  c64::new(0. , 0.) ,   c64::new(0.25 , 0.) , c64::new(0. , 0.)  ,  c64::new(0. , 0.)   ] ,
                                     [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0.25 , 0.) ,  c64::new(0. , 0.)   ] ,
                                     [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0.25 , 0.)   ] ];

for eta_index in 0..100{
    let mut eta = c64::new((eta_index as f64)/100. , 0.);
  
    for theta_index in 0..360{
        let mut theta: f64 = (theta_index as f64)*PI/180.;
        let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
        let mut rho_pure: MatrixC64 = create_dens_matrix(&psi_part_entangled);
 
        let mut rho_mixed = eta*rho_pure.clone() + (1.-eta)*mixed_state.clone();
        let mut concurrence: f64 = find_concurrence(rho_mixed);
        points.push( (theta, eta.re, concurrence) ); 
    }

}


let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Concurrence of mixed state and partially entangled state", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..2.*PI, 0_f64..1_f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_desc("theta")
        .y_desc("eta")
        .draw()?;
    
    chart.draw_series(
        points.into_iter()
            .map(|(theta, eta, concurrence)| {
                Rectangle::new(
                    [ (theta, eta) , (theta + PI/18., eta + 0.01) ], 
                    HSLColor(
                        240.0 / 360.0 - 240.0 / 360.0 * (concurrence),
                        1.0,
                        0.1 + 0.4 * concurrence,
                    ).filled(),
                )
            })
    );
    
    
    Ok(())
});

figure

*Fidelity*

This function is a distance measurement of two density matrices $\rho$ and $\sigma$. It is expressed as $F=tr\sqrt{\sqrt{\rho}\sigma\sqrt{\rho}}$. Here we graph a Fidelity over eta, where the pure state $\rho$ stays constant but $\eta$ is varied. The pure state, $\rho_{pure}$, is the density matrix of the Bell State $\psi_- = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$. The mixed state $\sigma$ = $\eta\rho_{pure} + (1-\eta)\rho_{mixed}$.

In [ ]:
let bell_psi_minus_coef: VecC64 = array![c64::new(0.0         , 0.0) , c64::new(norm_const , 0.0) , 
                                        c64::new(-norm_const , 0.0) , c64::new(0.0        , 0.0) ];

let mixed_state: MatrixC64 = array![ [  c64::new(0.25 , 0.) ,  c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0. , 0.)  ] , 
                                  [  c64::new(0. , 0.) ,   c64::new(0.25 , 0.) , c64::new(0. , 0.)  ,  c64::new(0. , 0.)   ] ,
                                  [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0.25 , 0.) ,  c64::new(0. , 0.)   ] ,
                                  [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0.25 , 0.)   ] ];

let mut points: Vec<(f64, f64)> = Vec::new();

for i in 0..100{
    let eta = c64::new((i as f64)/100. , 0.); 
    let mut rho_pure = create_dens_matrix(&bell_psi_minus_coef);
    let mut sigma_mixed = eta*rho_pure.clone() + (c64::new(1. , 0.) - eta)*mixed_state.clone();
    let mut fidelity = find_fidelity(rho_pure,sigma_mixed);
    points.push( (eta.re, fidelity) );
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Fidelity of sigma_mixed and rho_pure", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..1_f64, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("eta")
        .y_desc("Fidelity")
        .draw()?;
    chart.draw_series(points.into_iter().map(|(x,y)| Circle::new((x,y), 3, BLUE.filled())));
    Ok(())
});

figure

*Negativity*

The negativity is expressed as $\frac{||\rho^{\Gamma_{A}}||_{1}-1}{2}$, and it prodives a measure of entanglement between states. It ranges from 0 to $\frac{2}{d}$, where $d$ is the dimension of the matrix. $||\rho^{\Gamma_{A}}||_{1}$ is the trace norm of the partial transpose of $\rho$. 

In [ ]:
let mut points: Vec<(f64, f64, f64)> = Vec::new();

let mixed_state: MatrixC64 = array![ [  c64::new(0.25 , 0.) ,  c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0. , 0.)  ] , 
                                     [  c64::new(0. , 0.) ,   c64::new(0.25 , 0.) , c64::new(0. , 0.)  ,  c64::new(0. , 0.)   ] ,
                                     [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0.25 , 0.) ,  c64::new(0. , 0.)   ] ,
                                     [  c64::new(0. , 0.) ,   c64::new(0. , 0.)  , c64::new(0. , 0.)  ,  c64::new(0.25 , 0.)   ] ];

for eta_index in 0..100{
    let mut eta = c64::new((eta_index as f64)/100. , 0.);
  
    for theta_index in 0..360{
        let mut theta: f64 = (theta_index as f64)*PI/180.;
        let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
        let mut rho_pure: MatrixC64 = create_dens_matrix(&psi_part_entangled);
 
        let mut rho_mixed = eta*rho_pure.clone() + (1.-eta)*mixed_state.clone();
        let mut negativity: f64 = find_negativity(rho_mixed);
        points.push( (theta, eta.re, negativity) ); 
    }

}


let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Negativity of mixed state and partially entangled state", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..2.*PI, 0_f64..1_f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_desc("theta")
        .y_desc("eta")
        .draw()?;
    
    chart.draw_series(
        points.into_iter()
            .map(|(theta, eta, negativity)| {
                Rectangle::new(
                    [ (theta, eta) , (theta + PI/18., eta + 0.01) ], 
                    HSLColor(
                        240.0 / 360.0 - 240.0 / 360.0 * (negativity),
                        1.0,
                        0.5,
                    ).filled(),
                )
            })
    );
    
    
    Ok(())
});

figure